In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import numpy as np
import re
pd.set_option('display.max_colwidth', None)
from datetime import datetime
import ast

In [3]:
ufo = pd.read_csv('ufo.csv')

In [4]:
# convert date to date/time
ufo.date = pd.to_datetime(ufo.date)

<span style='background:purple; font-size:25px'>getting image links from reportlink</span>

# subset data on images = yes

In [28]:
# images_df = pd.DataFrame(images_list, columns=['reportlink', 'image_links']) # make dataframe with columns

<span style='background:purple; font-size:25px'>combine ufo and images_df dataframes</span>

In [34]:
# ufo = pd.merge(ufo, images_df, how = 'left', on = 'reportlink')

<span style='background:purple; font-size:25px'>combine ufo and lat, lng data (merge on location)</span>

In [14]:
lat_lng = pd.read_csv('ufo_columns_lat_lng.csv', encoding='latin1')

In [235]:
# lat_lng = lat_lng.drop('Unnamed: 5', axis=1)

In [15]:
lat_lng

,city,state,country,lat,lng
0,Gilbertsville,KY,usa,37.024495,-88.302493
1,Golden Pond (near),KY,usa,36.785608,-88.024192
2,Goshen/Prospect,KY,usa,38.403318,-85.583077
3,Gosport,KY,usa,37.839333,-84.270018
4,Grand Rives,KY,usa,37.003111,-88.234475
...,...,...,...,...,...
28327,Ft. Knox,KY,usa,37.883664,-85.965269
28328,Ft. Mitchell,KY,usa,39.047205,-84.559917
28329,Ft. Wright,KY,usa,39.051156,-84.535029
28330,Gamaliel,KY,usa,36.639778,-85.796644


In [18]:
ufo = pd.merge(ufo, lat_lng, how='left', on=['city', 'state'])

In [23]:
ufo.drop('country_y', axis = 1, inplace=True)

In [25]:
ufo.rename(columns={'country_x': 'country'}, inplace=True)

In [30]:
ufo.country = ufo.country.str.lower()

# save new df

In [32]:
ufo.to_csv('ufo.csv')

<span style='background:purple; font-size:25px'>getting distinct city names</span>

<span style='background:yellow; font-size:25px'>SCRATCH PAD</span>

In [65]:
ufo.month.isna().value_counts()

False    129293
Name: month, dtype: int64

In [90]:
listToStr = ' '.join([str(elem) for elem in enumerate (ufo.image_links)])


In [116]:
ufo.loc[ufo['images'] == 'NaN']

,date,month,year,reportlink,city,state,country,shape,duration,description,post_date,images,image_links


In [118]:
ufo.image_links = ufo.image_links.replace('nan', np.nan)

In [201]:
# make lists strings
for index, row in ufo.iterrows():
    if pd.notna(row['image_links']):
        ufo.loc[index, 'image_links'] = row['image_links'].strip('[]')

In [212]:
ufo['image_links'] = ufo['image_links'].str.replace("'", '') # remove single quotes


In [213]:
ufo.image_links[128248]

'https://nuforc.org/wp-content/uploads/wpforms/624-c824613929464448a7e062112379f845/20220405_195733-ca637737fb8d5d9ddd597a83e399a054.jpg, https://nuforc.org/wp-content/uploads/wpforms/624-c824613929464448a7e062112379f845/20220405_195709-dfbe62e82af00b489420bdc69baf43b9.jpg, https://nuforc.org/wp-content/uploads/wpforms/624-c824613929464448a7e062112379f845/20220405_195826-b18f8faa0d28213c1abf695ec5345bc2.jpg'

In [34]:
ufo.date.min()

Timestamp('1905-07-02 00:00:00')

In [38]:
testing

,date,month,year,reportlink,city,state,country,shape,duration,description,post_date,images,image_links,lat,lng
0,7/27/2023,July,2023,https://nuforc.org/webreports/reports/177/S177048.html,Plainfield,IN,usa,triangle,1 minute,2 objects above store.,7/29/2023,NaN,NaN,39.704212,-86.399439
1,7/26/2023,July,2023,https://nuforc.org/webreports/reports/177/S177053.html,Little Falls,MN,usa,triangle,2 minutes,Saw 2 to 4 triangle craft with bright white lights on the angle tips,7/29/2023,NaN,NaN,45.980556,-94.369444
2,7/25/2023,July,2023,https://nuforc.org/webreports/reports/177/S177081.html,sacramento,CA,usa,triangle,around 5 min,Silent-more of a slight gush of wind/ triangular object/maybe size of jet/maybe quarter of mile high??,7/29/2023,NaN,NaN,38.581572,-121.494400
3,7/23/2023,July,2023,https://nuforc.org/webreports/reports/176/S176930.html,Buford,GA,usa,triangle,3+ hours,3 triangle shape white with reddish tint hovering and moving in unison and hovering in the same spot for hours,7/29/2023,yes,https://nuforc.org/wp-content/uploads/wpforms/624-c824613929464448a7e062112379f845/IMG_9719-7cf0d35e0eede2f212f7d9a88a070cd7.jpeg,34.120656,-84.004351
4,7/22/2023,July,2023,https://nuforc.org/webreports/reports/176/S176927.html,Bellefonte,PA,usa,triangle,1 to 2 minutes,"Craft was low overhead, made no noise. Made almost a right angle turn, never slowing down. 3 blinking lights",7/29/2023,NaN,NaN,40.913394,-77.778335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129810,5/15/1967,May,1967,https://nuforc.org/webreports/reports/034/S34031.html,Belvedere,CA,usa,teardrop,Approx. 10 seconds,Teardrop outlined in white light with three white circular lights inside other wise dark interior of tear shape.,1/17/2004,NaN,NaN,37.872704,-122.464417
129811,6/1/1961,June,1961,https://nuforc.org/webreports/reports/052/S52151.html,NaN,NY,usa,teardrop,30 minutes,I was 11 yrs old. My Aunt and Uncle and I were driveing from Elmira NY to Bradford PA we were in the hills coming around a bend near a,10/30/2006,NaN,NaN,NaN,NaN
129812,7/10/1958,July,1958,https://nuforc.org/webreports/reports/027/S27374.html,Cheektowaga,NY,usa,teardrop,10 -20 secs,White teardrop shaped object observed during the daytime,3/21/2003,NaN,NaN,42.907148,-78.754289
129813,6/30/1958,June,1958,https://nuforc.org/webreports/reports/080/S80850.html,Conyers,GA,usa,teardrop,5 min.,Sighting of a UFO ca. 1958.,5/2/2011,NaN,NaN,33.667610,-84.017690


In [37]:
testing = pd.read_csv('ufo.csv')

In [41]:
testing.date.value_counts().head(20)

11/7/2015         206
7/4/2014          184
7/4/2010          184
11/16/1999        162
7/4/2012          158
7/4/2013          151
7/4/2011          138
7/4/2015          125
9/19/2009          99
11/7/2015 0:00     86
1/1/2014           85
7/4/2014 0:00      85
7/27/2016          83
10/31/2004         78
12/31/2013         77
7/6/2013           72
7/4/1997           68
7/5/2014           68
7/4/2009           66
12/31/2012         63
Name: date, dtype: int64

In [36]:
sheet_names = ['ufo', '']

,Unnamed: 0,date,month,year,reportlink,city,state,country,shape,duration,description,post_date,images,image_links,lat,lng
0,0,7/27/2023,July,2023,https://nuforc.org/webreports/reports/177/S177048.html,Plainfield,IN,usa,triangle,1 minute,2 objects above store.,7/29/2023,NaN,NaN,39.704212,-86.399439
1,1,7/26/2023,July,2023,https://nuforc.org/webreports/reports/177/S177053.html,Little Falls,MN,usa,triangle,2 minutes,Saw 2 to 4 triangle craft with bright white lights on the angle tips,7/29/2023,NaN,NaN,45.980556,-94.369444
2,2,7/25/2023,July,2023,https://nuforc.org/webreports/reports/177/S177081.html,sacramento,CA,usa,triangle,around 5 min,Silent-more of a slight gush of wind/ triangular object/maybe size of jet/maybe quarter of mile high??,7/29/2023,NaN,NaN,38.581572,-121.494400
3,3,7/23/2023,July,2023,https://nuforc.org/webreports/reports/176/S176930.html,Buford,GA,usa,triangle,3+ hours,3 triangle shape white with reddish tint hovering and moving in unison and hovering in the same spot for hours,7/29/2023,yes,https://nuforc.org/wp-content/uploads/wpforms/624-c824613929464448a7e062112379f845/IMG_9719-7cf0d35e0eede2f212f7d9a88a070cd7.jpeg,34.120656,-84.004351
4,4,7/22/2023,July,2023,https://nuforc.org/webreports/reports/176/S176927.html,Bellefonte,PA,usa,triangle,1 to 2 minutes,"Craft was low overhead, made no noise. Made almost a right angle turn, never slowing down. 3 blinking lights",7/29/2023,NaN,NaN,40.913394,-77.778335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129810,129810,5/15/1967,May,1967,https://nuforc.org/webreports/reports/034/S34031.html,Belvedere,CA,usa,teardrop,Approx. 10 seconds,Teardrop outlined in white light with three white circular lights inside other wise dark interior of tear shape.,1/17/2004,NaN,NaN,37.872704,-122.464417
129811,129811,6/1/1961,June,1961,https://nuforc.org/webreports/reports/052/S52151.html,NaN,NY,usa,teardrop,30 minutes,I was 11 yrs old. My Aunt and Uncle and I were driveing from Elmira NY to Bradford PA we were in the hills coming around a bend near a,10/30/2006,NaN,NaN,NaN,NaN
129812,129812,7/10/1958,July,1958,https://nuforc.org/webreports/reports/027/S27374.html,Cheektowaga,NY,usa,teardrop,10 -20 secs,White teardrop shaped object observed during the daytime,3/21/2003,NaN,NaN,42.907148,-78.754289
129813,129813,6/30/1958,June,1958,https://nuforc.org/webreports/reports/080/S80850.html,Conyers,GA,usa,teardrop,5 min.,Sighting of a UFO ca. 1958.,5/2/2011,NaN,NaN,33.667610,-84.017690
